#                                                Projet final du Kit Data Science 2021  

In [1]:
#Import
import urllib
import openpyxl 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import tempfile
from zipfile import ZipFile
import shutil
import os
from fnmatch import fnmatch
import re
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat
from sklearn import linear_model
%matplotlib notebook
#Option d'affichage 
pd.set_option("display.min_rows", 16)

### 1.Chargement et affichage des fichiers excels

In [2]:
url_classement = "https://www.vendeeglobe.org/fr/classement" 

def get_soup_from_url(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    return soup

In [3]:
def get_extension_from_url(url):
    soup = get_soup_from_url(url)
    dates_complet = soup.find_all("option")
    extension = [val["value"].strip()+'.xlsx' for val in dates_complet]
    return extension

In [4]:
#Definition des dates d'arrivées et nombres e fichiers à télécharger
Ensemble_extensions = get_extension_from_url(url_classement)
#Tri de l'ensemble pour ordonner le classement de la plus ancienne date à la plus récente
Ensemble_extensions.sort()
#Index du dernier fichier pris en compte
idr_dernierfichier = Ensemble_extensions.index('20210127_170000.xlsx')
#Ensemble_extensions[2:Ensemble_extensions.index('20210127_210000.xlsx')])
Ensemble_extensions=Ensemble_extensions[2:idr_dernierfichier]
#Le nombre de fichiers pris en compte
len(Ensemble_extensions)

485

In [5]:
#Telechargement des fichiers. Décommenter pour télécharger 
#for extension in Ensemble_extensions:
    #lien = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_"+extension
    #file_name = 'vendeeglobe_'+extension
    #urllib.request.urlretrieve(lien, file_name)

In [6]:
#Fonction qui convertit les données en fichiers en fichiers exploitables
def change_in_zip(file_name, name_filter, change):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)           
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)
    return 

In [7]:
#Fonction ui crée le dataframe des classements
def df_classement():
    df = pd.DataFrame()
    for extension in Ensemble_extensions:
        #date_enregistr = pd.DataFrame(extension,columns = ["Date_enregistr"], index = [np.arange(33)])
        #df = df.append(date_enregistr,ignore_index=True)
        filename ='vendeeglobe_'+extension
        change_in_zip(filename, 
              'xl/styles.xml',lambda d: re.sub(b'xxid="\d*"', b"", d.strip()))
        file_un = pd.read_excel(filename,header = 0,
                                    names = ["Num_linexcel", "Rang/Rank","Nat./Voile-Nat./Sail",
                                "Skipper/Bateau-Skipper/Crew","Heure FR-Hour FR","Latitude","Longitude",
                                "Depuis 30mn:Cap-Heading","Depuis 30mn:Vitess-Speed","Depuis 30mn:VMG-VMG",
                                "Depuis 30mn:Distance","Depuis dernier classement:Cap-Heading",
                                "Depuis dernier classement:Vitesse-Speed","Depuis dernier classement:VMG-VMG",
                                "Depuis dernier classement:Distance","Depuis 24H:Cap-Heading","Depuis 24H:Vitesse-Speed",
                                "Depuis 24H:VMG","Depuis 24H:Distance","DTF", "DTL"],
                                index_col = None)
        file_un =file_un.drop([0,1,2,3],axis = 0)
        file_un = file_un.drop([37,38,39,40],axis = 0)
        #df_unite = pd.merge(date_enregistr, file_un, left_index=True, right_index=True, how="left")
        df = df.append(file_un, ignore_index=True)
    return df

In [8]:
%%time
df_classement= df_classement()
df_classement.head()

CPU times: user 25.9 s, sys: 926 ms, total: 26.9 s
Wall time: 27.4 s


,Num_linexcel,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,...,Depuis dernier classement:Cap-Heading,Depuis dernier classement:Vitesse-Speed,Depuis dernier classement:VMG-VMG,Depuis dernier classement:Distance,Depuis 24H:Cap-Heading,Depuis 24H:Vitesse-Speed,Depuis 24H:VMG,Depuis 24H:Distance,DTF,DTL
0,NaN,1,\nFRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,...,357°,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm
1,NaN,2,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,...,357°,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm
2,NaN,3,\nFRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,...,357°,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm
3,NaN,4,\nFRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,...,357°,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm
4,NaN,5,\nFRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,...,357°,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm


### 2. Extraction des caractéristiques techniques de chacun des voiliers

In [9]:
# Page web avec les fiches techniques des voiliers du Vendée Globe
url_caract = "https://www.vendeeglobe.org/fr/glossaire" 

In [10]:
def get_fiche_tech_url(url):
    soup = get_soup_from_url(url)
    df_fiche_tech = pd.DataFrame()
    boats_list = soup.find_all("div",class_="boats-list__infos")
    for element in boats_list:
        ligne =[[element.find("h3", class_="boats-list__boat-name").text,
                 element.find("span", class_="boats-list__skipper-name").text]]
        df = pd.DataFrame(ligne, columns = ["Nom bateau","Nom skipper"])
        df_fiche_tech = df_fiche_tech.append(df,ignore_index=True)
    return df_fiche_tech

In [11]:
df_T1 = get_fiche_tech_url(url_caract).reset_index(drop = True)
print(df_T1)

                            Nom bateau           Nom skipper
0             NEWREST - ART & FENÊTRES        Fabrice AMEDEO
1                 PURE - Best Western®      Romain ATTANASIO
2                     TSE -  4MYPLANET        Alexia BARRIER
3                        Maître CoQ IV      Yannick BESTAVEN
4                               CHARAL         Jérémie BEYOU
5    LA MIE CÂLINE - ARTISANS ARTIPÔLE     Arnaud BOISSIÈRES
6                      BUREAU VALLEE 2          Louis BURTON
7                 ONE PLANET ONE OCEAN           Didac COSTA
8                         GROUPE SÉTIN         Manuel COUSIN
9                   BANQUE POPULAIRE X       Clarisse CREMER
10                              APIVIA         Charlie DALIN
11                   INITIATIVES-COEUR            Sam DAVIES
12                               MERCI   Sébastien DESTREMAU
13                OMIA - WATER FAMILY       Benjamin DUTREUX
14                                 PRB       Kevin ESCOFFIER
15           Compagnie d

### Autres caractéristiques techniques

In [12]:
def get_autre__fiche_tech_url(url):
    soup = get_soup_from_url(url)
    df_fiche_tech = pd.DataFrame()
    boats_list = soup.find_all("ul", class_="boats-list__popup-specs-list")
    for element in boats_list:
        ligne =[[element.select(".boats-list__popup-specs-list li")[0].text.split()[-2:],
                element.select(".boats-list__popup-specs-list li")[1].text.split()[5:],
                element.select(".boats-list__popup-specs-list li")[-12].text.split()[2:],
                element.select(".boats-list__popup-specs-list li")[-11].text.split()[3:],
                element.select(".boats-list__popup-specs-list li")[-10].text.split()[4:],
                element.select(".boats-list__popup-specs-list li")[-9].text.split()[2:],
                element.select(".boats-list__popup-specs-list li")[-8].text.split()[2:],
                element.select(".boats-list__popup-specs-list li")[-7].text.split()[3:],
                element.select(".boats-list__popup-specs-list li")[-6].text.split()[3:],
                element.select(".boats-list__popup-specs-list li")[-5].text.split()[-2:],
                element.select(".boats-list__popup-specs-list li")[-4].text.split()[-2],
                element.select(".boats-list__popup-specs-list li")[-3].text.split()[-1],
                element.select(".boats-list__popup-specs-list li")[-2].text.split()[-2],
                element.select(".boats-list__popup-specs-list li")[-1].text.split()[-2],]]
        df = pd.DataFrame(ligne, columns = ["Numéro de voile","Anciens noms du bateau","Architecte",
                                            "Chantier", "Date de lancement","Longueur","Largeur","Tirant d'eau",
                                            "Déplacement (poids)","Nombre de dérives","Hauteur mât","Voile quille",
                                           "Surface de voiles au près","Surface de voiles au portant"])
        df_fiche_tech = df_fiche_tech.append(df,ignore_index=True)
    return df_fiche_tech
    

In [13]:
df_T2=get_autre__fiche_tech_url(url_caract)
df_T2

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,"[FRA, 56]","[No, Way, Back,, Vento, di, Sardegna]",[VPLP/Verdier],[Marine],"[01, Août, 2015]","[18,28, m]","[5,85, m]","[4,50, m]","[7, t]","[:, foils]",29,monotype,320,570
1,"[FRA, 49]","[Gitana, Eighty,, Synerciel,, Newrest-Matmut]","[Bruce, Farr, Design]","[Ocean, Marine, (Nouvelle, Zélande)]","[08, Mars, 2007]","[18,28m]","[5,80m]","[4,50m]",[9t],"[:, 2]",:,forgé,280,560
2,"[:, FRA72]","[Famille, Mary-Etamine, du, Lys,, Initiatives,...","[Marc, Lombard]",[France],"[01, Mars, 1998]","[18,28m]","[5,54m]","[4,50m]",[9t],"[:, 2]",29,acier,260,580
3,"[:, 17]","[Safran, 2, -, Des, Voiles, et, Vous]","[Verdier, -, VPLP]",[Technologies],"[12, Mars, 2015]","[18,28, m]","[5,80, m]","[4,50, m]","[8, t]","[:, foils]",29,soudé,310,550
4,"[:, 08]",[],[VPLP],[Technologies],"[18, Août, 2018]","[18,28, m]","[5,85, m]","[4,50, m]",[8t],"[:, foils]",29,acier,320,600
5,"[FRA, 14]","[Ecover3,, Président,, Gamesa,, Kilcullen, Voy...","[Owen, Clarke, Design, LLP, -, Clay, Oliver]","[Marine, -, Mer, Agitée]","[03, Août, 2007]","[18,28, m]","[5,65, m]","[4,50, m]","[7,9, tonnes]","[:, foils]",29,vérin,300,610
6,"[:, 18]","[Banque, Populaire, VIII]","[Verdier, -, VPLP]",[Technologies],"[09, Juin, 2015]","[18,28, m]","[5,80, m]","[4,50, m]","[7,6, t]","[:, foils]",28,acier,300,600
7,"[ESP, 33]","[Kingfisher, -, Educacion, sin, Fronteras, -, ...","[Owen, Clarke, Design]",[Yachts],"[02, Février, 2000]","[18,28, m]","[5,30, m]","[4,50, m]","[8,9, t]","[:, 2]",26,acier,240,470
8,"[FRA, 71]","[Paprec-Virbac2,, Estrella, Damm,, We, are, Wa...","[Bruce, Farr, Yacht, Design]","[Ocean, Marine, (Nouvelle-Zélande)]","[02, Février, 2007]","[18,28, m]","[5,80, m]","[4,50, m]","[9, t]","[2, asymétriques]",:,hydraulique,270,560
9,"[:, FRA30]","[Macif, -, SMA]","[Verdier, -, VPLP]","[-, Mer, Agitée]","[01, Mars, 2011]","[18,28, m]","[5,70, m]","[4,5, m]","[7,7, t]","[:, 2]",29,forgé,340,570


In [14]:
# Mettre toutes les données des fiches techniques dans le même dataframe
df_ftech =pd.merge(df_T1, df_T2, left_index=True, right_index=True, how="outer")

df_ftech

,Nom bateau,Nom skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,NEWREST - ART & FENÊTRES,Fabrice AMEDEO,"[FRA, 56]","[No, Way, Back,, Vento, di, Sardegna]",[VPLP/Verdier],[Marine],"[01, Août, 2015]","[18,28, m]","[5,85, m]","[4,50, m]","[7, t]","[:, foils]",29,monotype,320,570
1,PURE - Best Western®,Romain ATTANASIO,"[FRA, 49]","[Gitana, Eighty,, Synerciel,, Newrest-Matmut]","[Bruce, Farr, Design]","[Ocean, Marine, (Nouvelle, Zélande)]","[08, Mars, 2007]","[18,28m]","[5,80m]","[4,50m]",[9t],"[:, 2]",:,forgé,280,560
2,TSE - 4MYPLANET,Alexia BARRIER,"[:, FRA72]","[Famille, Mary-Etamine, du, Lys,, Initiatives,...","[Marc, Lombard]",[France],"[01, Mars, 1998]","[18,28m]","[5,54m]","[4,50m]",[9t],"[:, 2]",29,acier,260,580
3,Maître CoQ IV,Yannick BESTAVEN,"[:, 17]","[Safran, 2, -, Des, Voiles, et, Vous]","[Verdier, -, VPLP]",[Technologies],"[12, Mars, 2015]","[18,28, m]","[5,80, m]","[4,50, m]","[8, t]","[:, foils]",29,soudé,310,550
4,CHARAL,Jérémie BEYOU,"[:, 08]",[],[VPLP],[Technologies],"[18, Août, 2018]","[18,28, m]","[5,85, m]","[4,50, m]",[8t],"[:, foils]",29,acier,320,600
5,LA MIE CÂLINE - ARTISANS ARTIPÔLE,Arnaud BOISSIÈRES,"[FRA, 14]","[Ecover3,, Président,, Gamesa,, Kilcullen, Voy...","[Owen, Clarke, Design, LLP, -, Clay, Oliver]","[Marine, -, Mer, Agitée]","[03, Août, 2007]","[18,28, m]","[5,65, m]","[4,50, m]","[7,9, tonnes]","[:, foils]",29,vérin,300,610
6,BUREAU VALLEE 2,Louis BURTON,"[:, 18]","[Banque, Populaire, VIII]","[Verdier, -, VPLP]",[Technologies],"[09, Juin, 2015]","[18,28, m]","[5,80, m]","[4,50, m]","[7,6, t]","[:, foils]",28,acier,300,600
7,ONE PLANET ONE OCEAN,Didac COSTA,"[ESP, 33]","[Kingfisher, -, Educacion, sin, Fronteras, -, ...","[Owen, Clarke, Design]",[Yachts],"[02, Février, 2000]","[18,28, m]","[5,30, m]","[4,50, m]","[8,9, t]","[:, 2]",26,acier,240,470
8,GROUPE SÉTIN,Manuel COUSIN,"[FRA, 71]","[Paprec-Virbac2,, Estrella, Damm,, We, are, Wa...","[Bruce, Farr, Yacht, Design]","[Ocean, Marine, (Nouvelle-Zélande)]","[02, Février, 2007]","[18,28, m]","[5,80, m]","[4,50, m]","[9, t]","[2, asymétriques]",:,hydraulique,270,560
9,BANQUE POPULAIRE X,Clarisse CREMER,"[:, FRA30]","[Macif, -, SMA]","[Verdier, -, VPLP]","[-, Mer, Agitée]","[01, Mars, 2011]","[18,28, m]","[5,70, m]","[4,5, m]","[7,7, t]","[:, 2]",29,forgé,340,570


### 3. Préparation des données 

#### 3.1 Préparation des données de classement

In [15]:
#On reprend les fichiers de classement en suprimant la colonne des NaN
#del df_classement["Num_linexcel"]
df_classement.head()

,Num_linexcel,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,...,Depuis dernier classement:Cap-Heading,Depuis dernier classement:Vitesse-Speed,Depuis dernier classement:VMG-VMG,Depuis dernier classement:Distance,Depuis 24H:Cap-Heading,Depuis 24H:Vitesse-Speed,Depuis 24H:VMG,Depuis 24H:Distance,DTF,DTL
0,NaN,1,\nFRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,...,357°,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm
1,NaN,2,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,...,357°,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm
2,NaN,3,\nFRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,...,357°,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm
3,NaN,4,\nFRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,...,357°,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm
4,NaN,5,\nFRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,...,357°,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm


In [16]:
df_classement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16005 entries, 0 to 16004
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Num_linexcel                             0 non-null      float64
 1   Rang/Rank                                16005 non-null  object 
 2   Nat./Voile-Nat./Sail                     16005 non-null  object 
 3   Skipper/Bateau-Skipper/Crew              16005 non-null  object 
 4   Heure FR-Hour FR                         13784 non-null  object 
 5   Latitude                                 13784 non-null  object 
 6   Longitude                                13784 non-null  object 
 7   Depuis 30mn:Cap-Heading                  13784 non-null  object 
 8   Depuis 30mn:Vitess-Speed                 13784 non-null  object 
 9   Depuis 30mn:VMG-VMG                      13784 non-null  object 
 10  Depuis 30mn:Distance                     13784

In [17]:
# La primère colonne n'a que des Nan. On l'a supprime 
del df_classement["Num_linexcel"]
#Restindex
df_classement.reset_index()
df_classement.head()

,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,Depuis 30mn:Distance,Depuis dernier classement:Cap-Heading,Depuis dernier classement:Vitesse-Speed,Depuis dernier classement:VMG-VMG,Depuis dernier classement:Distance,Depuis 24H:Cap-Heading,Depuis 24H:Vitesse-Speed,Depuis 24H:VMG,Depuis 24H:Distance,DTF,DTL
0,1,\nFRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,357°,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm
1,2,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,357°,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm
2,3,\nFRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,0.5 nm,357°,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm
3,4,\nFRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,0.7 nm,357°,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm
4,5,\nFRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,0.2 nm,357°,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm


In [18]:
# lignes dupliquées
df_classement.duplicated().value_counts()

False    13714
True      2291
dtype: int64

In [19]:
df_classement.apply(len, axis=0)

Rang/Rank                                  16005
Nat./Voile-Nat./Sail                       16005
Skipper/Bateau-Skipper/Crew                16005
Heure FR-Hour FR                           16005
Latitude                                   16005
Longitude                                  16005
Depuis 30mn:Cap-Heading                    16005
Depuis 30mn:Vitess-Speed                   16005
Depuis 30mn:VMG-VMG                        16005
Depuis 30mn:Distance                       16005
Depuis dernier classement:Cap-Heading      16005
Depuis dernier classement:Vitesse-Speed    16005
Depuis dernier classement:VMG-VMG          16005
Depuis dernier classement:Distance         16005
Depuis 24H:Cap-Heading                     16005
Depuis 24H:Vitesse-Speed                   16005
Depuis 24H:VMG                             16005
Depuis 24H:Distance                        16005
DTF                                        16005
DTL                                        16005
dtype: int64

In [20]:

df_classement["Nat./Voile-Nat./Sail"] = df_classement["Nat./Voile-Nat./Sail"].str.strip()


In [21]:
df_classement.head()

,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,Depuis 30mn:Distance,Depuis dernier classement:Cap-Heading,Depuis dernier classement:Vitesse-Speed,Depuis dernier classement:VMG-VMG,Depuis dernier classement:Distance,Depuis 24H:Cap-Heading,Depuis 24H:Vitesse-Speed,Depuis 24H:VMG,Depuis 24H:Distance,DTF,DTL
0,1,FRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,357°,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm
1,2,MON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,357°,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm
2,3,FRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,0.5 nm,357°,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm
3,4,FRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,0.7 nm,357°,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm
4,5,FRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,0.2 nm,357°,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm


In [22]:
# 3) Rajoutez 2 colonnes "Opérateur", "Type Couverture" et "Réseau"
def data_addcolumns():
    df = df_classement
    df["Prenom skipper"] = df["Skipper/Bateau-Skipper/Crew"].apply(lambda x: x.split("\n")[0])
    df["Prenom skipper"] = df["Prenom skipper"].apply(lambda x: x.split()[0])
    return df

In [23]:
df_classement = data_addcolumns().copy()
df_classement.head()

,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,Depuis 30mn:Distance,...,Depuis dernier classement:Vitesse-Speed,Depuis dernier classement:VMG-VMG,Depuis dernier classement:Distance,Depuis 24H:Cap-Heading,Depuis 24H:Vitesse-Speed,Depuis 24H:VMG,Depuis 24H:Distance,DTF,DTL,Prenom skipper
0,1,FRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,...,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm,Louis
1,2,MON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,...,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm,Boris
2,3,FRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,0.5 nm,...,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm,Jérémie
3,4,FRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,0.7 nm,...,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm,Thomas
4,5,FRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,0.2 nm,...,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm,Maxime


In [24]:

df_classement["Skipper/Bateau-Skipper/Crew"] = df_classement["Skipper/Bateau-Skipper/Crew"].str.replace("\n"," ").astype(str)                                                                                   

                                                                                           

In [25]:
df_classement["Heure FR-Hour FR"] = df_classement["Heure FR-Hour FR"].str.replace("FR\n"," ").astype(str) 

In [26]:
df_classement["Depuis 30mn:Cap-Heading"] = df_classement["Depuis 30mn:Cap-Heading"].str.replace("°"," ").astype(str)


In [27]:
df_classement["Depuis 30mn:Vitess-Speed"] = df_classement["Depuis 30mn:Vitess-Speed"].str.replace("kts"," ").astype(float)


In [28]:
df_classement["Depuis 30mn:VMG-VMG"] = df_classement["Depuis 30mn:VMG-VMG"].str.replace("kts"," ").astype(float)


In [29]:
df_classement["Depuis 30mn:Distance"] = df_classement["Depuis 30mn:Distance"].str.replace("nm"," ").astype(float)


In [30]:
df_classement["Depuis dernier classement:Cap-Heading"] = df_classement["Depuis dernier classement:Cap-Heading"].str.replace("°"," ").astype(float)



In [31]:
df_classement["Depuis dernier classement:Vitesse-Speed"] = df_classement["Depuis dernier classement:Vitesse-Speed"].str.replace("kts"," ").astype(float)


In [32]:
df_classement["Depuis dernier classement:VMG-VMG"] = df_classement["Depuis dernier classement:VMG-VMG"].str.replace("kts"," ").astype(float)



In [33]:
df_classement["Depuis dernier classement:Distance"] = df_classement["Depuis dernier classement:Distance"].str.replace("nm"," ").astype(float)



In [34]:
df_classement["Depuis 24H:Cap-Heading"] = df_classement["Depuis 24H:Cap-Heading"].str.replace("°"," ").astype(float) 

In [35]:
df_classement["Depuis 24H:Vitesse-Speed"] = df_classement["Depuis 24H:Vitesse-Speed"].str.replace("kts"," ").astype(float)
 

In [36]:
df_classement["Depuis 24H:VMG"] = df_classement["Depuis 24H:VMG"].str.replace("kts"," ").astype(float)


In [37]:
df_classement["Depuis 24H:Distance"] = df_classement["Depuis 24H:Distance"].str.replace("nm"," ").astype(float)


In [38]:
df_classement["DTF"] = df_classement["DTF"].str.replace("nm"," ").astype(float)

In [39]:
df_classement["DTL"] = df_classement["DTL"].str.replace("nm"," ").astype(float)

In [40]:
df_classement.head()

,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,Depuis 30mn:Distance,...,Depuis dernier classement:Vitesse-Speed,Depuis dernier classement:VMG-VMG,Depuis dernier classement:Distance,Depuis 24H:Cap-Heading,Depuis 24H:Vitesse-Speed,Depuis 24H:VMG,Depuis 24H:Distance,DTF,DTL,Prenom skipper
0,1,FRA 18,Louis Burton Bureau Vallée 2,15:30,46°24.46'N,01°50.48'W,241,17.7,17.5,0.3,...,0.0,0.0,2788.0,201.0,0.3,0.3,6.1,24293.9,0.0,Louis
1,2,MON 10,Boris Herrmann Seaexplorer - Yacht Club De Monaco,15:31 1min,46°24.34'N,01°49.82'W,241,11.1,10.9,0.4,...,0.0,0.0,2787.9,196.0,0.3,0.2,6.0,24294.2,0.4,Boris
2,3,FRA 8,Jérémie Beyou Charal,15:30,46°24.91'N,01°49.99'W,244,15.5,15.5,0.5,...,0.0,0.0,2788.5,199.0,0.2,0.2,5.5,24294.3,0.5,Jérémie
3,4,FRA 59,Thomas Ruyant LinkedOut,15:30,46°24.71'N,01°49.68'W,244,13.2,13.1,0.7,...,0.0,0.0,2788.3,196.0,0.2,0.2,5.6,24294.5,0.6,Thomas
4,5,FRA 53,Maxime Sorel V And B Mayenne,15:30,46°24.59'N,01°49.56'W,246,10.9,10.9,0.2,...,0.0,0.0,2788.1,195.0,0.8,0.7,5.8,24294.5,0.6,Maxime


#### 3.2 Préparation des données techniques

In [41]:
df_ftech

,Nom bateau,Nom skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,NEWREST - ART & FENÊTRES,Fabrice AMEDEO,"[FRA, 56]","[No, Way, Back,, Vento, di, Sardegna]",[VPLP/Verdier],[Marine],"[01, Août, 2015]","[18,28, m]","[5,85, m]","[4,50, m]","[7, t]","[:, foils]",29,monotype,320,570
1,PURE - Best Western®,Romain ATTANASIO,"[FRA, 49]","[Gitana, Eighty,, Synerciel,, Newrest-Matmut]","[Bruce, Farr, Design]","[Ocean, Marine, (Nouvelle, Zélande)]","[08, Mars, 2007]","[18,28m]","[5,80m]","[4,50m]",[9t],"[:, 2]",:,forgé,280,560
2,TSE - 4MYPLANET,Alexia BARRIER,"[:, FRA72]","[Famille, Mary-Etamine, du, Lys,, Initiatives,...","[Marc, Lombard]",[France],"[01, Mars, 1998]","[18,28m]","[5,54m]","[4,50m]",[9t],"[:, 2]",29,acier,260,580
3,Maître CoQ IV,Yannick BESTAVEN,"[:, 17]","[Safran, 2, -, Des, Voiles, et, Vous]","[Verdier, -, VPLP]",[Technologies],"[12, Mars, 2015]","[18,28, m]","[5,80, m]","[4,50, m]","[8, t]","[:, foils]",29,soudé,310,550
4,CHARAL,Jérémie BEYOU,"[:, 08]",[],[VPLP],[Technologies],"[18, Août, 2018]","[18,28, m]","[5,85, m]","[4,50, m]",[8t],"[:, foils]",29,acier,320,600
5,LA MIE CÂLINE - ARTISANS ARTIPÔLE,Arnaud BOISSIÈRES,"[FRA, 14]","[Ecover3,, Président,, Gamesa,, Kilcullen, Voy...","[Owen, Clarke, Design, LLP, -, Clay, Oliver]","[Marine, -, Mer, Agitée]","[03, Août, 2007]","[18,28, m]","[5,65, m]","[4,50, m]","[7,9, tonnes]","[:, foils]",29,vérin,300,610
6,BUREAU VALLEE 2,Louis BURTON,"[:, 18]","[Banque, Populaire, VIII]","[Verdier, -, VPLP]",[Technologies],"[09, Juin, 2015]","[18,28, m]","[5,80, m]","[4,50, m]","[7,6, t]","[:, foils]",28,acier,300,600
7,ONE PLANET ONE OCEAN,Didac COSTA,"[ESP, 33]","[Kingfisher, -, Educacion, sin, Fronteras, -, ...","[Owen, Clarke, Design]",[Yachts],"[02, Février, 2000]","[18,28, m]","[5,30, m]","[4,50, m]","[8,9, t]","[:, 2]",26,acier,240,470
8,GROUPE SÉTIN,Manuel COUSIN,"[FRA, 71]","[Paprec-Virbac2,, Estrella, Damm,, We, are, Wa...","[Bruce, Farr, Yacht, Design]","[Ocean, Marine, (Nouvelle-Zélande)]","[02, Février, 2007]","[18,28, m]","[5,80, m]","[4,50, m]","[9, t]","[2, asymétriques]",:,hydraulique,270,560
9,BANQUE POPULAIRE X,Clarisse CREMER,"[:, FRA30]","[Macif, -, SMA]","[Verdier, -, VPLP]","[-, Mer, Agitée]","[01, Mars, 2011]","[18,28, m]","[5,70, m]","[4,5, m]","[7,7, t]","[:, 2]",29,forgé,340,570


In [42]:
df_ftech["Numéro de voile"] = df_ftech["Numéro de voile"].apply(lambda x:" ".join(map(str, x))).str.replace(":"," ").astype(str)



In [43]:
df_ftech["Anciens noms du bateau"] = df_ftech["Anciens noms du bateau"].apply(lambda x:" ".join(map(str, x)))

In [44]:
df_ftech["Architecte"] = df_ftech["Architecte"].apply(lambda x:" ".join(map(str, x)))


In [45]:
df_ftech["Chantier"] = df_ftech["Chantier"].apply(lambda x:" ".join(map(str, x)))

In [46]:
df_ftech["Date de lancement"] = df_ftech["Date de lancement"].apply(lambda x:" ".join(map(str, x)))

In [47]:
df_ftech["Longueur"] = df_ftech["Longueur"].apply(lambda x:" ".join(map(str, x))).str.replace("m"," ").astype(str)


In [48]:
df_ftech["Longueur"] = df_ftech["Longueur"].apply(lambda x:" ".join(map(str, x))).str.replace(" , ",".").astype(str)


In [49]:
df_ftech["Largeur"] = df_ftech["Largeur"].apply(lambda x:" ".join(map(str, x))).str.replace("m"," ").astype(str)


In [50]:
df_ftech["Tirant d\'eau"] = df_ftech["Tirant d\'eau"].apply(lambda x:" ".join(map(str, x))).str.replace("m"," ").astype(str)


In [51]:

df_ftech["Déplacement (poids)"] = df_ftech["Déplacement (poids)"].apply(lambda x:" ".join(map(str, x))).str.replace("t"," ").astype(str)
df_ftech["Déplacement (poids)"] = df_ftech["Déplacement (poids)"].str.replace("onnes"," ").astype(str)


In [52]:

df_ftech["Nombre de dérives"] = df_ftech["Nombre de dérives"].apply(lambda x:" ".join(map(str, x))).str.replace(":"," ").astype(str)


In [53]:
# Dernière ligne du dataFrame à la main

df_ftech.iloc[32] =  ["CORUM L'EPARGNE","Nicolas TROUSSEL","FR 6","  ","Juan Kouyoumdjian","CDK technologies- Mer Agitée","15 Mai 2020","18,28","5,70","4,5","7,9","foils","27,30","  ","270","535"]



In [54]:
df_ftech.head()

,Nom bateau,Nom skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,NEWREST - ART & FENÊTRES,Fabrice AMEDEO,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Marine,01 Août 2015,1 8.2 8,"5,85","4,50",7,foils,29,monotype,320,570
1,PURE - Best Western®,Romain ATTANASIO,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Ocean Marine (Nouvelle Zélande),08 Mars 2007,1 8.2 8,"5,80","4,50",9,2,:,forgé,280,560
2,TSE - 4MYPLANET,Alexia BARRIER,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,France,01 Mars 1998,1 8.2 8,"5,54","4,50",9,2,29,acier,260,580
3,Maître CoQ IV,Yannick BESTAVEN,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,Technologies,12 Mars 2015,1 8.2 8,"5,80","4,50",8,foils,29,soudé,310,550
4,CHARAL,Jérémie BEYOU,08,,VPLP,Technologies,18 Août 2018,1 8.2 8,"5,85","4,50",8,foils,29,acier,320,600


In [55]:
# 3) Rajoutez 1 colonne "Opérateur", "Type Couverture" et "Réseau"
def data_addcolumns2():
    df = df_ftech
    df["Prenom skipper"] = df["Nom skipper"].apply(lambda x: x.split(" ")[0])
    #df["Prenomm skipper"] = df["Prenom skipper"].apply(lambda x: x.split()[0])
    return df

In [56]:
df_ftech = data_addcolumns2().copy()

In [57]:
cols_time = ["Date de lancement"]
df_ftech[cols_time].head()

,Date de lancement
0,01 Août 2015
1,08 Mars 2007
2,01 Mars 1998
3,12 Mars 2015
4,18 Août 2018


### 4. Rapprochement des données 

In [58]:
#Fichier regroupant les classements et les aractéristiques chniques
df = pd.merge(df_classement, df_ftech, how='right',left_on='Prenom skipper',right_on='Prenom skipper')
df = df.sort_values(by = "Rang/Rank")

In [59]:
df.head()

,Rang/Rank,Nat./Voile-Nat./Sail,Skipper/Bateau-Skipper/Crew,Heure FR-Hour FR,Latitude,Longitude,Depuis 30mn:Cap-Heading,Depuis 30mn:Vitess-Speed,Depuis 30mn:VMG-VMG,Depuis 30mn:Distance,...,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
15078,1,GBR 99,Alex Thomson HUGO BOSS,11:30,21°53.68'N,29°48.82'W,187,15.6,14.9,7.8,...,15 Août 2019,1 8.2 8,"5,40","4,50","7,7",foils,29,forgé,330,630
1789,1,FRA 17,Yannick Bestaven Maître Coq IV,17:30,57°11.80'S,65°44.14'W,74,18.4,15.3,9.2,...,12 Mars 2015,1 8.2 8,"5,80","4,50",8,foils,29,soudé,310,550
1788,1,FRA 17,Yannick Bestaven Maître Coq IV,14:30,57°25.10'S,67°23.42'W,79,18.8,15.0,1.6,...,12 Mars 2015,1 8.2 8,"5,80","4,50",8,foils,29,soudé,310,550
1787,1,FRA 17,Yannick Bestaven Maître Coq IV,11:30,57°34.28'S,69°08.85'W,89,18.1,12.6,9.0,...,12 Mars 2015,1 8.2 8,"5,80","4,50",8,foils,29,soudé,310,550
1786,1,FRA 17,Yannick Bestaven Maître Coq IV,08:30,57°38.27'S,70°52.98'W,87,17.7,14.6,8.9,...,12 Mars 2015,1 8.2 8,"5,80","4,50",8,foils,29,soudé,310,550


In [60]:
df = df.dropna()

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14199 entries, 15078 to 9441
Data columns (total 37 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Rang/Rank                                14199 non-null  object 
 1   Nat./Voile-Nat./Sail                     14199 non-null  object 
 2   Skipper/Bateau-Skipper/Crew              14199 non-null  object 
 3   Heure FR-Hour FR                         14199 non-null  object 
 4   Latitude                                 14199 non-null  object 
 5   Longitude                                14199 non-null  object 
 6   Depuis 30mn:Cap-Heading                  14199 non-null  object 
 7   Depuis 30mn:Vitess-Speed                 14199 non-null  float64
 8   Depuis 30mn:VMG-VMG                      14199 non-null  float64
 9   Depuis 30mn:Distance                     14199 non-null  float64
 10  Depuis dernier classement:Cap-Heading    14

In [62]:
df.nunique

<bound method DataFrame.nunique of       Rang/Rank Nat./Voile-Nat./Sail     Skipper/Bateau-Skipper/Crew  \
15078         1               GBR 99          Alex Thomson HUGO BOSS   
1789          1               FRA 17  Yannick Bestaven Maître Coq IV   
1788          1               FRA 17  Yannick Bestaven Maître Coq IV   
1787          1               FRA 17  Yannick Bestaven Maître Coq IV   
1786          1               FRA 17  Yannick Bestaven Maître Coq IV   
1785          1               FRA 17  Yannick Bestaven Maître Coq IV   
1784          1               FRA 17  Yannick Bestaven Maître Coq IV   
1790          1               FRA 17  Yannick Bestaven Maître Coq IV   
...         ...                  ...                             ...   
9443          9               FRA 27          Isabelle Joschke MACSF   
9436          9               FRA 27          Isabelle Joschke MACSF   
9437          9               FRA 27          Isabelle Joschke MACSF   
9438          9              

In [63]:
X = pd.to_numeric(df["Depuis 24H:VMG"])
X = X.to_numpy()
Y = pd.to_numeric(df["Rang/Rank"])
Y = Y.to_numpy()
X_0 = pd.to_numeric(df["Depuis dernier classement:VMG-VMG"])
X_0 = X_0.to_numpy()
df_by_rang = df.groupby(["Rang/Rank","Skipper/Bateau-Skipper/Crew"])
df.shape[0]


14199

In [64]:
df_by_rang.size()

Rang/Rank  Skipper/Bateau-Skipper/Crew                            
1          Alex Thomson HUGO BOSS                                      46
           Benjamin Dutreux OMIA - Water Family                         1
           Charlie Dalin APIVIA                                       225
           Damien Seguin Groupe APICIL                                  5
           Jean Le Cam Yes we Cam !                                     9
           Jérémie Beyou Charal                                         5
           Louis Burton Bureau Vallée 2                                 7
           Maxime Sorel V And B Mayenne                                 8
                                                                     ... 
9          Maxime Sorel V And B Mayenne                                41
           Nicolas Troussel CORUM L'Épargne                             2
           Romain Attanasio Pure - Best Western Hotels and Resorts      5
           Stéphane Le Diraison Time For Ocea

### 5. Quelques statistiques

In [ ]:
df_by_rang['Depuis 24H:Cap-Heading'].plot(kind="hist")# Répartion par VMG
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
df_by_rang['Depuis 24H:VMG'].plot(kind="hist")

In [ ]:
df_by_rang['Depuis 30mn:VMG-VMG'].plot(kind="hist")

In [ ]:
#df_by_rang[["Depuis 24H:VMG","Depuis 30mn:VMG-VMG","Depuis dernier classement:VMG-VMG"]].plot(kind="hist")

In [ ]:
linmod = linear_model.LinearRegression()
linmod.fit(X.reshape(-1, 1), Y)

print('En utilisant ''sklearn.linear_model'', les paramètres estimés sont:')
print('b0 = {:.4f},'.format(linmod.intercept_))
print('b1 = {:.4f}.'.format(linmod.coef_[0]))

print('\nCoefficient de détermination R^2 = {:.4f}.'.format(linmod.score(X.reshape(-1, 1), Y)))



####  Ce coefficient de précision très faible montre que le modèle est très peu précis

#### Passons à la regression linéaire groupée

In [ ]:
Y_pred = linmod.predict(X.reshape(-1, 1))
plt.figure()
plt.scatter(X, Y, label= "Rang/Rank")
plt.plot(X, Y_pred, c='k', label='Regression line')
plt.title("Corrélation et régression linéaire entre le classement (rang) et la vitesse utile (VMG) ")
plt.xlabel("Depuis 24H:VMG")
plt.ylabel("Rang/Rank")
plt.legend()
plt.show()